In [ ]:
import numpy as np
from skfuzzy import control as ctrl
from skfuzzy import membership as mf

In [ ]:
# Initialize inputs (Antecedents)
PM25 = ctrl.Antecedent(np.arange(0, 501, 1), 'PM2.5')  # PM2.5 concentration in µg/m³
PM10 = ctrl.Antecedent(np.arange(0, 801, 1), 'PM10')  # PM10 concentration in µg/m³
NO2 = ctrl.Antecedent(np.arange(0, 501, 1), 'NO2')  # NO2 concentration in µg/m³
SO2 = ctrl.Antecedent(np.arange(0, 801, 1), 'SO2')  # SO2 concentration in µg/m³
CO = ctrl.Antecedent(np.arange(0, 31, 0.1), 'CO')  # CO concentration in µg/m³
O3 = ctrl.Antecedent(np.arange(0, 501, 1), 'O3')  # O3 concentration in µg/m³

# Define fuzzy membership functions for PM2.5
PM25['Very Low'] = mf.trapmf(PM25.universe, [0, 0, 5, 10])
PM25['Low'] = mf.trimf(PM25.universe, [5, 10, 20])
PM25['Moderate'] = mf.trimf(PM25.universe, [25, 55, 75])
PM25['High'] = mf.trimf(PM25.universe, [60, 100, 150])
PM25['Very High'] = mf.trapmf(PM25.universe, [120, 150, 300, 500])

# Define fuzzy membership functions for PM10
PM10['Very Low'] = mf.trapmf(PM10.universe, [0, 0, 10, 20])
PM10['Low'] = mf.trimf(PM10.universe, [15, 25, 50])
PM10['Moderate'] = mf.trimf(PM10.universe, [60, 120, 180])
PM10['High'] = mf.trimf(PM10.universe, [150, 250, 350])
PM10['Very High'] = mf.trapmf(PM10.universe, [300, 350, 500, 800])

# Define fuzzy membership functions for NO2
NO2['Very Low'] = mf.trapmf(NO2.universe, [0, 0, 10, 20])
NO2['Low'] = mf.trimf(NO2.universe, [15, 25, 40])
NO2['Moderate'] = mf.trimf(NO2.universe, [50, 90, 120])
NO2['High'] = mf.trimf(NO2.universe, [100, 160, 220])
NO2['Very High'] = mf.trapmf(NO2.universe, [200, 220, 300, 500])

# Define fuzzy membership functions for SO2
SO2['Very Low'] = mf.trapmf(SO2.universe, [0, 0, 10, 20])
SO2['Low'] = mf.trimf(SO2.universe, [15, 25, 40])
SO2['Moderate'] = mf.trimf(SO2.universe, [50, 100, 150])
SO2['High'] = mf.trimf(SO2.universe, [120, 200, 300])
SO2['Very High'] = mf.trapmf(SO2.universe, [250, 300, 500, 800])

# Define fuzzy membership functions for CO
CO['Very Low'] = mf.trapmf(CO.universe, [0, 0, 0.5, 1])
CO['Low'] = mf.trimf(CO.universe, [0.8, 1.2, 2])
CO['Moderate'] = mf.trimf(CO.universe, [2.5, 5, 7])
CO['High'] = mf.trimf(CO.universe, [6, 10, 15])
CO['Very High'] = mf.trapmf(CO.universe, [12, 15, 20, 30])

# Define fuzzy membership functions for O3
O3['Very Low'] = mf.trapmf(O3.universe, [0, 0, 20, 30])
O3['Low'] = mf.trimf(O3.universe, [25, 35, 50])
O3['Moderate'] = mf.trimf(O3.universe, [60, 100, 130])
O3['High'] = mf.trimf(O3.universe, [110, 160, 200])
O3['Very High'] = mf.trapmf(O3.universe, [180, 200, 300, 500])

In [ ]:
# Initialize output (Consequent): AQI
AQI = ctrl.Consequent(np.arange(0, 1001, 1), 'AQI')

# Initialize the linguistic output: AQI Category
AQI_Category = ctrl.Consequent(np.arange(0, 1001, 1), 'AQI_Category')

# Define fuzzy membership functions for AQI Category (trapezoidal)
AQI_Category['Good'] = mf.trapmf(AQI_Category.universe, [0, 0, 30, 60])
AQI_Category['Moderate'] = mf.trapmf(AQI_Category.universe, [40, 55, 85, 110])
AQI_Category['Unhealthy_for_Sensitive_Groups'] = mf.trapmf(AQI_Category.universe, [90, 105, 135, 160])
AQI_Category['Unhealthy'] = mf.trapmf(AQI_Category.universe, [140, 160, 185, 210])
AQI_Category['Very_Unhealthy'] = mf.trapmf(AQI_Category.universe, [195, 210, 255, 320])
AQI_Category['Hazardous'] = mf.trapmf(AQI_Category.universe, [300, 320, 500, 1000])

In [ ]:
# ---------------------
# Define the fuzzy rules
# ---------------------

# Rule 1: If PM2.5 is Low AND NO2 is Low, THEN AQI is Good
rule1 = ctrl.Rule(PM25['Low'] & NO2['Low'], AQI_Category['Good'])

# Rule 2: If PM2.5 is Moderate AND PM10 is Moderate, THEN AQI is Moderate
rule2 = ctrl.Rule(PM25['Moderate'] & PM10['Moderate'], AQI_Category['Moderate'])

# Rule 3: If PM2.5 is High OR NO2 is High, THEN AQI is Unhealthy for Sensitive Groups
rule3 = ctrl.Rule(PM25['High'] | NO2['High'], AQI_Category['Unhealthy_for_Sensitive_Groups'])

# Rule 4: If PM2.5 is High AND NO2 is High, THEN AQI is Unhealthy
rule4 = ctrl.Rule(PM25['High'] & NO2['High'], AQI_Category['Unhealthy'])

# Rule 5: If PM2.5 is High AND NO2 is Very High, THEN AQI is Very Unhealthy
rule5 = ctrl.Rule(PM25['High'] & NO2['Very High'], AQI_Category['Very_Unhealthy'])

# Rule 6: If PM2.5, NO2, and CO are Very High, THEN AQI is Hazardous
rule6 = ctrl.Rule(PM25['Very High'] & NO2['Very High'] & CO['Very High'], AQI_Category['Hazardous'])

## Optional extended rules to capture worsening combined pollution
## (If one pollutant is very dominant, output shifts to higher severity)

## Rule 7: If PM is High AND NO2 is Moderate, THEN AQI is Unhealthy for Sensitive Groups
#rule7 = ctrl.Rule(PM['High'] & NO2['Moderate'], AQI_Category['Unhealthy_for_Sensitive_Groups'])

## Rule 8: If PM is Moderate AND NO2 is High, THEN AQI is Unhealthy for Sensitive Groups
#rule8 = ctrl.Rule(PM['Moderate'] & NO2['High'], AQI_Category['Unhealthy_for_Sensitive_Groups'])

# ---------------------
# Combine all rules into a list
# ---------------------
rules = [rule1, rule2, rule3, rule4, rule5, rule6]
